In [6]:
import torch
from torch import Tensor
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

### Cora Data Set
see https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.Planetoid

In [8]:
dataset = Planetoid(root='.', name='Cora')

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, cached=True,
                             normalize=True)
        self.conv2 = GCNConv(hidden_channels, out_channels, cached=True,
                             normalize=True)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

model = GCN(dataset.num_features, 16, dataset.num_classes)

In [9]:
model

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [11]:
data = dataset[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = model.to(device), data.to(device)

In [12]:
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.

In [13]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_weight)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)

In [14]:
@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_weight).argmax(dim=-1)

    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

In [17]:
best_val_acc = final_test_acc = 0
epochs=2000
for epoch in range(0, epochs):
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f"Epoch={epoch+1}, Loss={loss}, Train={train_acc}, Val={val_acc}, Test={test_acc}")

Epoch=1, Loss=0.03572746738791466, Train=1.0, Val=0.764, Test=0.798
Epoch=2, Loss=0.027080627158284187, Train=1.0, Val=0.766, Test=0.795
Epoch=3, Loss=0.054363250732421875, Train=1.0, Val=0.764, Test=0.795
Epoch=4, Loss=0.04991898313164711, Train=1.0, Val=0.766, Test=0.795
Epoch=5, Loss=0.05219227075576782, Train=1.0, Val=0.768, Test=0.8
Epoch=6, Loss=0.04436289146542549, Train=1.0, Val=0.77, Test=0.803
Epoch=7, Loss=0.02423805743455887, Train=1.0, Val=0.772, Test=0.803
Epoch=8, Loss=0.0315953828394413, Train=1.0, Val=0.774, Test=0.801
Epoch=9, Loss=0.05530337244272232, Train=1.0, Val=0.774, Test=0.801
Epoch=10, Loss=0.023088684305548668, Train=1.0, Val=0.782, Test=0.804
Epoch=11, Loss=0.03182704746723175, Train=1.0, Val=0.776, Test=0.804
Epoch=12, Loss=0.04353439062833786, Train=1.0, Val=0.776, Test=0.804
Epoch=13, Loss=0.04713058099150658, Train=1.0, Val=0.776, Test=0.804
Epoch=14, Loss=0.046349626034498215, Train=1.0, Val=0.774, Test=0.804
Epoch=15, Loss=0.0391673818230629, Train=1.

Epoch=128, Loss=0.02938162535429001, Train=1.0, Val=0.758, Test=0.807
Epoch=129, Loss=0.030716337263584137, Train=1.0, Val=0.756, Test=0.807
Epoch=130, Loss=0.021251151338219643, Train=1.0, Val=0.762, Test=0.807
Epoch=131, Loss=0.021193480119109154, Train=1.0, Val=0.766, Test=0.807
Epoch=132, Loss=0.04299963265657425, Train=1.0, Val=0.772, Test=0.807
Epoch=133, Loss=0.02870912291109562, Train=1.0, Val=0.772, Test=0.807
Epoch=134, Loss=0.020981866866350174, Train=1.0, Val=0.772, Test=0.807
Epoch=135, Loss=0.016179589554667473, Train=1.0, Val=0.77, Test=0.807
Epoch=136, Loss=0.026384038850665092, Train=1.0, Val=0.768, Test=0.807
Epoch=137, Loss=0.03094840794801712, Train=1.0, Val=0.77, Test=0.807
Epoch=138, Loss=0.026016319170594215, Train=1.0, Val=0.77, Test=0.807
Epoch=139, Loss=0.022624244913458824, Train=1.0, Val=0.778, Test=0.807
Epoch=140, Loss=0.04215715825557709, Train=1.0, Val=0.78, Test=0.807
Epoch=141, Loss=0.017967654392123222, Train=1.0, Val=0.782, Test=0.807
Epoch=142, Loss

Epoch=255, Loss=0.0607202872633934, Train=1.0, Val=0.772, Test=0.807
Epoch=256, Loss=0.025865629315376282, Train=1.0, Val=0.766, Test=0.807
Epoch=257, Loss=0.02914176695048809, Train=1.0, Val=0.758, Test=0.807
Epoch=258, Loss=0.026154903694987297, Train=1.0, Val=0.762, Test=0.807
Epoch=259, Loss=0.040770016610622406, Train=1.0, Val=0.76, Test=0.807
Epoch=260, Loss=0.04886345937848091, Train=1.0, Val=0.766, Test=0.807
Epoch=261, Loss=0.024148600175976753, Train=1.0, Val=0.766, Test=0.807
Epoch=262, Loss=0.022515039891004562, Train=1.0, Val=0.766, Test=0.807
Epoch=263, Loss=0.027237530797719955, Train=1.0, Val=0.77, Test=0.807
Epoch=264, Loss=0.02138066664338112, Train=1.0, Val=0.77, Test=0.807
Epoch=265, Loss=0.025189444422721863, Train=1.0, Val=0.77, Test=0.807
Epoch=266, Loss=0.022241294384002686, Train=1.0, Val=0.782, Test=0.807
Epoch=267, Loss=0.02186308056116104, Train=1.0, Val=0.778, Test=0.807
Epoch=268, Loss=0.030065208673477173, Train=1.0, Val=0.782, Test=0.807
Epoch=269, Loss=

Epoch=382, Loss=0.02873491495847702, Train=1.0, Val=0.748, Test=0.807
Epoch=383, Loss=0.02155459113419056, Train=1.0, Val=0.746, Test=0.807
Epoch=384, Loss=0.020460888743400574, Train=1.0, Val=0.75, Test=0.807
Epoch=385, Loss=0.026946336030960083, Train=1.0, Val=0.746, Test=0.807
Epoch=386, Loss=0.015713194385170937, Train=1.0, Val=0.748, Test=0.807
Epoch=387, Loss=0.024823546409606934, Train=1.0, Val=0.756, Test=0.807
Epoch=388, Loss=0.023833325132727623, Train=1.0, Val=0.76, Test=0.807
Epoch=389, Loss=0.04478349909186363, Train=1.0, Val=0.766, Test=0.807
Epoch=390, Loss=0.03244169056415558, Train=1.0, Val=0.774, Test=0.807
Epoch=391, Loss=0.01292932964861393, Train=1.0, Val=0.778, Test=0.807
Epoch=392, Loss=0.013050869107246399, Train=1.0, Val=0.778, Test=0.807
Epoch=393, Loss=0.019755607470870018, Train=1.0, Val=0.78, Test=0.807
Epoch=394, Loss=0.012502972967922688, Train=1.0, Val=0.78, Test=0.807
Epoch=395, Loss=0.020172564312815666, Train=1.0, Val=0.78, Test=0.807
Epoch=396, Loss=

Epoch=505, Loss=0.02098066359758377, Train=1.0, Val=0.764, Test=0.807
Epoch=506, Loss=0.02836127020418644, Train=1.0, Val=0.762, Test=0.807
Epoch=507, Loss=0.015201235190033913, Train=1.0, Val=0.762, Test=0.807
Epoch=508, Loss=0.038573283702135086, Train=1.0, Val=0.764, Test=0.807
Epoch=509, Loss=0.02880648523569107, Train=1.0, Val=0.762, Test=0.807
Epoch=510, Loss=0.014180236496031284, Train=1.0, Val=0.762, Test=0.807
Epoch=511, Loss=0.016018101945519447, Train=1.0, Val=0.754, Test=0.807
Epoch=512, Loss=0.019505048170685768, Train=1.0, Val=0.748, Test=0.807
Epoch=513, Loss=0.017711691558361053, Train=1.0, Val=0.75, Test=0.807
Epoch=514, Loss=0.03764842823147774, Train=1.0, Val=0.756, Test=0.807
Epoch=515, Loss=0.012526057660579681, Train=1.0, Val=0.758, Test=0.807
Epoch=516, Loss=0.03838512673974037, Train=1.0, Val=0.758, Test=0.807
Epoch=517, Loss=0.02671477198600769, Train=1.0, Val=0.758, Test=0.807
Epoch=518, Loss=0.021533746272325516, Train=1.0, Val=0.762, Test=0.807
Epoch=519, Lo

...

Epoch=1482, Loss=0.022638825699687004, Train=1.0, Val=0.786, Test=0.807
Epoch=1483, Loss=0.022831030189990997, Train=1.0, Val=0.782, Test=0.807
Epoch=1484, Loss=0.014879448339343071, Train=1.0, Val=0.776, Test=0.807
Epoch=1485, Loss=0.03966489061713219, Train=1.0, Val=0.77, Test=0.807
Epoch=1486, Loss=0.026540514081716537, Train=1.0, Val=0.77, Test=0.807
Epoch=1487, Loss=0.012105941772460938, Train=1.0, Val=0.776, Test=0.807
Epoch=1488, Loss=0.014329951256513596, Train=1.0, Val=0.778, Test=0.807
Epoch=1489, Loss=0.016919663175940514, Train=1.0, Val=0.782, Test=0.807
Epoch=1490, Loss=0.029570722952485085, Train=1.0, Val=0.778, Test=0.807
Epoch=1491, Loss=0.005104699172079563, Train=1.0, Val=0.774, Test=0.807
Epoch=1492, Loss=0.017845643684267998, Train=1.0, Val=0.774, Test=0.807
Epoch=1493, Loss=0.014155266806483269, Train=1.0, Val=0.776, Test=0.807
Epoch=1494, Loss=0.009632227011024952, Train=1.0, Val=0.774, Test=0.807
Epoch=1495, Loss=0.013646099716424942, Train=1.0, Val=0.776, Test=0

Epoch=1597, Loss=0.056023478507995605, Train=1.0, Val=0.752, Test=0.807
Epoch=1598, Loss=0.008570109494030476, Train=1.0, Val=0.75, Test=0.807
Epoch=1599, Loss=0.055782217532396317, Train=1.0, Val=0.744, Test=0.807
Epoch=1600, Loss=0.01673431135714054, Train=1.0, Val=0.744, Test=0.807
Epoch=1601, Loss=0.039659686386585236, Train=1.0, Val=0.746, Test=0.807
Epoch=1602, Loss=0.008403000421822071, Train=1.0, Val=0.75, Test=0.807
Epoch=1603, Loss=0.008468271233141422, Train=1.0, Val=0.744, Test=0.807
Epoch=1604, Loss=0.025915050879120827, Train=1.0, Val=0.744, Test=0.807
Epoch=1605, Loss=0.02224881947040558, Train=1.0, Val=0.746, Test=0.807
Epoch=1606, Loss=0.0138250682502985, Train=1.0, Val=0.746, Test=0.807
Epoch=1607, Loss=0.024395275861024857, Train=1.0, Val=0.744, Test=0.807
Epoch=1608, Loss=0.019353974610567093, Train=1.0, Val=0.742, Test=0.807
Epoch=1609, Loss=0.010971475392580032, Train=1.0, Val=0.75, Test=0.807
Epoch=1610, Loss=0.018177473917603493, Train=1.0, Val=0.752, Test=0.807

Epoch=1715, Loss=0.007886392995715141, Train=1.0, Val=0.774, Test=0.807
Epoch=1716, Loss=0.019019488245248795, Train=1.0, Val=0.774, Test=0.807
Epoch=1717, Loss=0.04145025461912155, Train=1.0, Val=0.774, Test=0.807
Epoch=1718, Loss=0.01692551001906395, Train=1.0, Val=0.772, Test=0.807
Epoch=1719, Loss=0.006539784837514162, Train=1.0, Val=0.774, Test=0.807
Epoch=1720, Loss=0.08293482661247253, Train=1.0, Val=0.774, Test=0.807
Epoch=1721, Loss=0.04756828397512436, Train=1.0, Val=0.774, Test=0.807
Epoch=1722, Loss=0.003659726120531559, Train=1.0, Val=0.776, Test=0.807
Epoch=1723, Loss=0.0063128843903541565, Train=1.0, Val=0.772, Test=0.807
Epoch=1724, Loss=0.01261887140572071, Train=1.0, Val=0.77, Test=0.807
Epoch=1725, Loss=0.023569244891405106, Train=1.0, Val=0.768, Test=0.807
Epoch=1726, Loss=0.01063874363899231, Train=1.0, Val=0.764, Test=0.807
Epoch=1727, Loss=0.03411957249045372, Train=1.0, Val=0.764, Test=0.807
Epoch=1728, Loss=0.012071045115590096, Train=1.0, Val=0.76, Test=0.807


Epoch=1830, Loss=0.01765117608010769, Train=1.0, Val=0.756, Test=0.807
Epoch=1831, Loss=0.009110425598919392, Train=1.0, Val=0.758, Test=0.807
Epoch=1832, Loss=0.028500191867351532, Train=1.0, Val=0.756, Test=0.807
Epoch=1833, Loss=0.015194864012300968, Train=1.0, Val=0.754, Test=0.807
Epoch=1834, Loss=0.015915149822831154, Train=1.0, Val=0.754, Test=0.807
Epoch=1835, Loss=0.010195822454988956, Train=1.0, Val=0.752, Test=0.807
Epoch=1836, Loss=0.01903308555483818, Train=1.0, Val=0.754, Test=0.807
Epoch=1837, Loss=0.014159154146909714, Train=1.0, Val=0.758, Test=0.807
Epoch=1838, Loss=0.027290649712085724, Train=1.0, Val=0.768, Test=0.807
Epoch=1839, Loss=0.009686962701380253, Train=1.0, Val=0.762, Test=0.807
Epoch=1840, Loss=0.030918052420020103, Train=1.0, Val=0.76, Test=0.807
Epoch=1841, Loss=0.006996823940426111, Train=1.0, Val=0.764, Test=0.807
Epoch=1842, Loss=0.014999529346823692, Train=1.0, Val=0.764, Test=0.807
Epoch=1843, Loss=0.015463088639080524, Train=1.0, Val=0.762, Test=0

Epoch=1948, Loss=0.026012737303972244, Train=1.0, Val=0.752, Test=0.807
Epoch=1949, Loss=0.014268307946622372, Train=1.0, Val=0.752, Test=0.807
Epoch=1950, Loss=0.00740260723978281, Train=1.0, Val=0.754, Test=0.807
Epoch=1951, Loss=0.01484525203704834, Train=1.0, Val=0.76, Test=0.807
Epoch=1952, Loss=0.015204086899757385, Train=1.0, Val=0.76, Test=0.807
Epoch=1953, Loss=0.009975377470254898, Train=1.0, Val=0.758, Test=0.807
Epoch=1954, Loss=0.03125984966754913, Train=1.0, Val=0.762, Test=0.807
Epoch=1955, Loss=0.005742434877902269, Train=1.0, Val=0.76, Test=0.807
Epoch=1956, Loss=0.013767356984317303, Train=1.0, Val=0.764, Test=0.807
Epoch=1957, Loss=0.0034489731770008802, Train=1.0, Val=0.764, Test=0.807
Epoch=1958, Loss=0.014955191873013973, Train=1.0, Val=0.764, Test=0.807
Epoch=1959, Loss=0.029695341363549232, Train=1.0, Val=0.766, Test=0.807
Epoch=1960, Loss=0.007055478636175394, Train=1.0, Val=0.762, Test=0.807
Epoch=1961, Loss=0.006072315853089094, Train=1.0, Val=0.76, Test=0.80